<a href="https://colab.research.google.com/github/mzkhan2000/COLAB/blob/main/Job_title_similarity_usingKNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# check out if google dride mount suceessful 
!ls "/content/drive/My Drive/Data Scientist Study Case "

ads-50k.json


In [10]:
# dataset file path in the google dride
filename = '/content/drive/My Drive/Data Scientist Study Case /ads-50k.json'

In [57]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [58]:
ads_df = pd.read_json(filename, lines=True)

In [12]:
ads_df.head()

,id,title,abstract,content,metadata
0,38915469,Recruitment Consultant,We are looking for someone to focus purely on ...,<HTML><p>Are you looking to join a thriving bu...,{'standout': {'bullet1': 'Join a Sector that i...
1,38934839,Computers Salesperson - Coburg,Passionate about exceptional customer service?...,<HTML><p>&middot;&nbsp;&nbsp;Casual hours as r...,{'additionalSalaryText': 'Attractive Commissio...
2,38946054,Senior Developer | SA,Readifarians are known for discovering the lat...,<HTML><p>Readify helps organizations innovate ...,"{'standout': {'bullet1': 'Design, develop, tes..."
3,38833950,Senior Commercial Property Manager | Leading T...,~ Rare opportunity for a Senior PM to step int...,<HTML><p><strong>WayPoint Recruitment&nbsp;</s...,{'additionalSalaryText': '$140k + Car Park - C...
4,38856271,Technology Manager | Travel Industry,Rare opportunity for an experienced Technology...,<HTML>This is a key role within a market leadi...,{'standout': {'bullet1': 'Lead overarching str...


In [63]:
ads_df['feature'] = ads_df['title'] + " " + ads_df['abstract']+ " " + ads_df['content'] 

In [64]:
import re
import string
# define a function for removing unnecessary/special characters and return a lower case plain texts/ Monir
def word_drop(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W", " ", text)
    text = re.sub('https?://\S+ |www\.\S+', '', text)
    text = re.sub('<.*?>', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text
# ----------------

In [65]:
ads_df['feature'] = ads_df['feature'].apply(word_drop) 

In [66]:
#initializing tfidf vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
#fitting and transforming the vector
tfidf_job = tfidf_vectorizer.fit_transform((ads_df['feature'])) 

In [67]:
features = tfidf_vectorizer.get_feature_names()
#tfidf_vectorizer.get_feature_names(): Array mapping from feature integer indices to feature name
#Get the row that belongs to Bill Clinton
row = ads_df[ads_df.title=='Senior Developer | SA'].index.tolist()[0]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [68]:
from sklearn.neighbors import NearestNeighbors
knn_neighbours = NearestNeighbors(n_neighbors=20)
knn_neighbours.fit(tfidf_job)

NearestNeighbors(n_neighbors=20)

In [69]:
closest_job = knn_neighbours.kneighbors(tfidf_job.getrow(row), return_distance=True)

In [70]:
print(closest_job[1][0])

[    2 34173 21033 19435 15626 32468  4315 24512 46289 39638   100 16412
 13946 41612  2546 16495 35455 19007 20373 36931]


In [71]:
names_index = closest_job[1][0]
names = [ads_df.iloc[row]['title'] for row in names_index]
distances = closest_job[0][0]

In [72]:
#If it is the case, we delete the name that has distance zero.
if distances[0]==0.0: distances,names = np.delete(distances,0),np.delete(names,0) 
data = pd.DataFrame({'Distances': distances,'Neighbours':names })

In [73]:
data.head(10)

,Distances,Neighbours
0,0.000000,Senior Developer | SA
1,0.061081,Senior Developer / Consultant | WA
2,0.984071,Lead Software Developer
3,0.991851,Senior Full Stack Software Engineer
4,0.993662,Senior Engineer - .Net Core/Angular 7 - Large ...
5,0.995907,Junior Software Engineer
6,0.999717,"Leader - Application Development, Support and ..."
7,1.003136,Senior Software Engineer
8,1.004869,Software Engineer
9,1.010843,Senior Technical Business Analyst
